# FIA0590 Prepaid Revenue Forecast

# Load libraries & global settings

In [1]:
%load_ext lab_black
%reload_ext lab_black

In [2]:
import pandas as pd
import numpy as np
import pandas_profiling
from datetime import datetime, timedelta

from pathlib import Path
import seaborn as sns
import datetime
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.figure_factory as ff


# Specific libraries for the task at hand:
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.forecasting.stl import STLForecast

# Stationary tests to find d
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

# AutoCorrelation Function and Partial AutoCorrelation Function
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


# Personal tools
from FIA0590_analysis_tools import (
    stl_fitting,
    plot_stl_fitting,
    plot_stacked_timeseries,
)

In [3]:
import dtale

In [4]:
TODAY = pd.Timestamp.today()
print(TODAY)

2022-03-02 13:48:56.372185


In [5]:
WORK_DIR = Path.cwd()
ANALYSIS_DIR = WORK_DIR / "analysis"
DATA_DIR = WORK_DIR / "data"

print(WORK_DIR)
print(ANALYSIS_DIR)
print(DATA_DIR)

C:\Users\truongs\Documents\GitHub\FIA0590_prepaid_revenue_forecast
C:\Users\truongs\Documents\GitHub\FIA0590_prepaid_revenue_forecast\analysis
C:\Users\truongs\Documents\GitHub\FIA0590_prepaid_revenue_forecast\data


In [6]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

# Load data

In [7]:
# What files are around:
files = list(DATA_DIR.iterdir())
for x in files:
    print(str(x)[len(str(DATA_DIR)) :])

\FIA0590_raw_data_20220215.csv


In [8]:
raw_prepaid_df = pd.read_csv(
    filepath_or_buffer=DATA_DIR / "FIA0590_raw_data_20220215.csv", sep=";"
).astype(
    {
        "Subscription_Type_Name": "category",
        "Subscription_Type_Desc": "category",
        "Brand_Name": "category",
    }
)

raw_prepaid_df["Calendar_Year_Month_Date"] = pd.to_datetime(
    raw_prepaid_df.Calendar_Year_Month_Date, format="%d.%m.%Y"
)

In [9]:
raw_prepaid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532 entries, 0 to 531
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Calendar_Year_Month_Id    532 non-null    int64         
 1   Calendar_Year_Month_Date  532 non-null    datetime64[ns]
 2   Subscription_Type_Name    532 non-null    category      
 3   Subscription_Type_Desc    532 non-null    category      
 4   Brand_Name                532 non-null    category      
 5   days_in_month             532 non-null    int64         
 6   rev_net_total             532 non-null    float64       
 7   subs_count                532 non-null    int64         
dtypes: category(3), datetime64[ns](1), float64(1), int64(3)
memory usage: 23.1 KB


# Data Preparation

In [10]:
prepaid_df = raw_prepaid_df

In [11]:
prepaid_df["rev_net_total_per_day"] = (
    prepaid_df.rev_net_total / prepaid_df.days_in_month
)

In [12]:
prepaid_df.sort_values("Calendar_Year_Month_Date", ascending=False)

,Calendar_Year_Month_Id,Calendar_Year_Month_Date,Subscription_Type_Name,Subscription_Type_Desc,Brand_Name,days_in_month,rev_net_total,subs_count,rev_net_total_per_day
0,202201,2022-01-01,MVNO4,Aldi Mobile Prepaid Subscription,Aldi,31,497451.79,48315,16046.83
133,202201,2022-01-01,MVNO3,Lebara Mobile Prepaid Subscription,Lebara,31,956734.19,95128,30862.39
399,202201,2022-01-01,CONGA,Yallo Mobile Prepaid Subscription,Yallo,31,1146105.08,153160,36971.13
266,202201,2022-01-01,Mobile Prepaid,Sunrise Mobile Prepaid Subscription - PRONTO -...,Sunrise,31,1237618.33,186343,39923.17
400,202112,2021-12-01,CONGA,Yallo Mobile Prepaid Subscription,Yallo,31,1178721.17,150937,38023.26
...,...,...,...,...,...,...,...,...,...
397,201102,2011-02-01,Mobile Prepaid,Sunrise Mobile Prepaid Subscription - PRONTO -...,Sunrise,28,8218698.29,453698,293524.94
132,201101,2011-01-01,MVNO4,Aldi Mobile Prepaid Subscription,Aldi,31,154360.77,19273,4979.38
265,201101,2011-01-01,MVNO3,Lebara Mobile Prepaid Subscription,Lebara,31,9735737.29,182358,314056.04
398,201101,2011-01-01,Mobile Prepaid,Sunrise Mobile Prepaid Subscription - PRONTO -...,Sunrise,31,9296977.94,474692,299902.51


In [13]:
prepaid_df.shape

(532, 9)

In [14]:
prepaid_df.describe()

,Calendar_Year_Month_Id,days_in_month,rev_net_total,subs_count,rev_net_total_per_day
count,532.00,532.00,532.00,532.00,532.00
mean,201610.97,30.44,3722343.12,222670.34,122301.82
std,319.51,0.81,3328510.64,140148.15,109292.21
min,201101.00,28.00,140741.11,18341.00,4979.38
25%,201310.00,30.00,867728.19,83303.75,27991.23
50%,201607.00,31.00,2559761.25,227022.50,84090.79
75%,201904.00,31.00,5914208.67,330803.50,193367.15
max,202201.00,31.00,11370017.26,563174.00,374699.20


# Data Exploration

## Pandas Profiling

Possible bug: https://github.com/pandas-profiling/pandas-profiling/issues/911

In [15]:
ANALYSIS_DIR
profile = prepaid_df.profile_report(
    title="FIA0590 Prepaid Revenue",
    minimal=True,  # explorative=True, # currently broken
)
_link_to_file_ = ANALYSIS_DIR / "FIA0590_profiling.html"

profile.to_file(output_file=_link_to_file_)

print(_link_to_file_)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\truongs\Documents\GitHub\FIA0590_prepaid_revenue_forecast\analysis\FIA0590_profiling.html


## D-Tale

In [17]:
dtale.show(prepaid_df, open_browser=True)

## Stacked Timeseries

In [21]:
plot_stacked_timeseries(plot_df=prepaid_df, ANALYSIS_DIR=ANALYSIS_DIR)

C:\Users\truongs\Documents\GitHub\FIA0590_prepaid_revenue_forecast\analysis\FIA0590_stacked_revenue.html


# STATSMODELS: _Seasonal-Trend decomposition_ (STD) using _Locally Estimated Scatterplot Smoothing_ (LOESS)

https://www.statsmodels.org/dev/examples/notebooks/generated/stl_decomposition.html

## Create dedicated time series dataframes for Sunrise, Yallo, and Lebara

In [22]:
print("\n SUNRISE \n")

sunrise_df = (
    prepaid_df[prepaid_df.Brand_Name.isin(["Sunrise"])]
    .sort_values("Calendar_Year_Month_Date")
    .reset_index()
    .drop(columns="index")
).set_index("Calendar_Year_Month_Date")

print(sunrise_df.info())

print("\n YALLO \n")

yallo_df = (
    prepaid_df[prepaid_df.Brand_Name.isin(["Yallo"])]
    .sort_values("Calendar_Year_Month_Date")
    .reset_index()
    .drop(columns="index")
).set_index("Calendar_Year_Month_Date")

print(yallo_df.info())

print("\n LEBARA \n")

lebara_df = (
    prepaid_df[prepaid_df.Brand_Name.isin(["Lebara"])]
    .sort_values("Calendar_Year_Month_Date")
    .reset_index()
    .drop(columns="index")
).set_index("Calendar_Year_Month_Date")

print(yallo_df.info())


 SUNRISE 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 133 entries, 2011-01-01 to 2022-01-01
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Calendar_Year_Month_Id  133 non-null    int64   
 1   Subscription_Type_Name  133 non-null    category
 2   Subscription_Type_Desc  133 non-null    category
 3   Brand_Name              133 non-null    category
 4   days_in_month           133 non-null    int64   
 5   rev_net_total           133 non-null    float64 
 6   subs_count              133 non-null    int64   
 7   rev_net_total_per_day   133 non-null    float64 
dtypes: category(3), float64(2), int64(3)
memory usage: 7.2 KB
None

 YALLO 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 133 entries, 2011-01-01 to 2022-01-01
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   Calendar_Ye

## Anomally Detection: Robust Fitting 

```
Revenue = Trend + Saisonalität + Fehler

        = LOESS + SARIMA       + Residuals
        
        = "Locally Estimated Scatterplot Smoothing" 
                    + "Seasonal Auto-Regressive Integrated Moving Average" 
                                + Residuals
                            
        = "Lokale polynomiale Regression (z.B. Marktsätigung)"
                    + "Saisonalitäten, welche sich über die Zeit verändern (z.b. sinkende Oszillation)" 
                            + "Fehler/Anomalie (z.B. Noise, Bundesmassnahmen, Systemfehler, )"
```

In [23]:
for (temp_df, brand) in zip(
    [sunrise_df, yallo_df, lebara_df], ["Sunrise", "Yallo", "Lebara"]
):

    res_df = stl_fitting(
        observation_df=temp_df,
        target_column="rev_net_total",
    )

    display(res_df.tail(3))
    plot_stl_fitting(res_df=res_df, brand_name=brand, ANALYSIS_DIR=ANALYSIS_DIR)
    print("\n")

,observed,trend,seasonal,residuals,weights
Calendar_Year_Month_Date,,,,,
2021-11-01,1183868.60,1415291.20,-62474.23,-168948.37,0.00
2021-12-01,1343088.60,1392728.12,-40681.21,-8958.31,0.99
2022-01-01,1237618.33,1370133.81,-122506.30,-10009.18,0.98


C:\Users\truongs\Documents\GitHub\FIA0590_prepaid_revenue_forecast\analysis\FIA0590_Sunrise_decomposition.html




,observed,trend,seasonal,residuals,weights
Calendar_Year_Month_Date,,,,,
2021-11-01,1226125.10,1235206.84,-3894.48,-5187.25,1.00
2021-12-01,1178721.17,1218893.66,-38936.67,-1235.83,1.00
2022-01-01,1146105.08,1202575.31,-52142.07,-4328.16,1.00


C:\Users\truongs\Documents\GitHub\FIA0590_prepaid_revenue_forecast\analysis\FIA0590_Yallo_decomposition.html




,observed,trend,seasonal,residuals,weights
Calendar_Year_Month_Date,,,,,
2021-11-01,1066201.76,1074231.89,3919.25,-11949.37,0.99
2021-12-01,994736.84,1065596.64,-38054.99,-32804.81,0.95
2022-01-01,956734.19,1056755.68,-66647.07,-33374.42,0.95


C:\Users\truongs\Documents\GitHub\FIA0590_prepaid_revenue_forecast\analysis\FIA0590_Lebara_decomposition.html




## Forecasting

### Parameter Optimisation: Stationarity and detrending (ADF/KPSS)

https://www.statsmodels.org/dev/examples/notebooks/generated/stationarity_detrending_adf_kpss.html?highlight=stationary%20test

In [24]:
diff_first_degree = (sunrise_df.rev_net_total.shift(-1) - sunrise_df.rev_net_total)[:-1]

In [25]:
diff_first_degree.tail()

Calendar_Year_Month_Date
2021-08-01    -29243.83
2021-09-01   -591344.47
2021-10-01    148811.82
2021-11-01    159220.00
2021-12-01   -105470.27
Name: rev_net_total, dtype: float64

In [26]:
diff_second_degree = (diff_first_degree.shift(-1) - diff_first_degree)[:-1]

In [27]:
def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)


def kpss_test(timeseries):
    print("Results of KPSS Test:")
    kpsstest = kpss(timeseries, regression="c", nlags="auto")
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    print(kpss_output)

In [28]:
adf_test(diff_first_degree)
print("\b")
kpss_test(diff_first_degree)

Results of Dickey-Fuller Test:
Test Statistic                 -1.12
p-value                         0.71
#Lags Used                     12.00
Number of Observations Used   119.00
Critical Value (1%)            -3.49
Critical Value (5%)            -2.89
Critical Value (10%)           -2.58
dtype: float64

Results of KPSS Test:
Test Statistic          0.44
p-value                 0.06
Lags Used               4.00
Critical Value (10%)    0.35
Critical Value (5%)     0.46
Critical Value (2.5%)   0.57
Critical Value (1%)     0.74
dtype: float64


_remark_: diff_first_degree are none-stationary based on both tests:

    1. Dicky-Fuller p-value: 0.68 far greater than 0.025 -> NO stationarity
    2. KPSS p-value: 0.02 smaller than 0.025 -> NO stationarity

In [29]:
adf_test(diff_second_degree)
print("\b")
kpss_test(diff_second_degree)

Results of Dickey-Fuller Test:
Test Statistic                -10.57
p-value                         0.00
#Lags Used                     11.00
Number of Observations Used   119.00
Critical Value (1%)            -3.49
Critical Value (5%)            -2.89
Critical Value (10%)           -2.58
dtype: float64

Results of KPSS Test:
Test Statistic           0.34
p-value                  0.10
Lags Used               49.00
Critical Value (10%)     0.35
Critical Value (5%)      0.46
Critical Value (2.5%)    0.57
Critical Value (1%)      0.74
dtype: float64


c:\program files\python\python38\lib\site-packages\statsmodels\tsa\stattools.py:2022: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.




_remark_: diff_second_degree are none-stationary based on both tests:

1. Dicky-Fuller p-value: 0.0 far greater than 0.025 ->  stationary
2. KPSS p-value: 0.1 bigger than 0.025  -> stationary

`Hence: d=2`

### Parameter Optimisation: ACF & PACF: Figuring out p and q

In [35]:
import matplotlib.pyplot as plt

In [58]:
fig, ax = plt.subplots(figsize=(20, 10))
plot_acf(diff_second_degree, lags=12, ax=ax)
_link_to_file_ = ANALYSIS_DIR / "acf.png"
plt.savefig(_link_to_file_)
print(_link_to_file_)

fig, ax = plt.subplots(figsize=(20, 10))
plot_pacf(diff_second_degree, lags=12)
_link_to_file_ = ANALYSIS_DIR / "pacf.png"
plt.savefig(_link_to_file_)
print(_link_to_file_)

C:\Users\truongs\Documents\GitHub\FIA0590_prepaid_revenue_forecast\analysis\acf.png
C:\Users\truongs\Documents\GitHub\FIA0590_prepaid_revenue_forecast\analysis\pacf.png


c:\program files\python\python38\lib\site-packages\statsmodels\graphics\tsaplots.py:348: FutureWarning:

The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.



Contenders for p are 1, 3, 4, 6, and 9

Contenders for q are 1, 2, 8, 9, 10, 11

### Forecasting with STL using Auto-Regressive Integrated Moving Average (ARIMA)

In [59]:
upper_training_limit = 202109  ## YYYYMM, limit is included

In [60]:
def create_forecasting_df(
    sunrise_df: pd.DataFrame, upper_training_limit=202109
) -> pd.DataFrame:

    sunrise_df.index.freq = sunrise_df.index.inferred_freq

    _filter_ = sunrise_df.Calendar_Year_Month_Id <= upper_training_limit
    _training_data_ = sunrise_df[_filter_].rev_net_total

    stlf = STLForecast(
        _training_data_,
        model=ARIMA,
        model_kwargs=dict(order=(1, 2, 1)),  # (p,d,q)
        period=12,  # 12 months periodicity
        robust=False,
    )

    stlf_res = stlf.fit()
    # forecast = stlf_res.forecast(steps=12)

    stlf_get_pred = stlf_res.get_prediction(start=sunrise_df.index.min(), end=TODAY)

    prediction_df = pd.DataFrame(
        {
            "predicted_mean": stlf_get_pred.predicted_mean,
            "var_pred_mean": stlf_get_pred.var_pred_mean,
        }
    )

    sunrise_with_prediction_df = pd.merge(
        right=prediction_df,
        left=sunrise_df,
        left_index=True,
        right_index=True,
        how="right",
    )
    sunrise_with_prediction_df = pd.merge(
        left=sunrise_with_prediction_df,
        right=stlf_get_pred.conf_int(alpha=0.10),
        left_index=True,
        right_index=True,
        how="left",
    )

    sunrise_with_prediction_df["residuals"] = (
        sunrise_with_prediction_df.rev_net_total
        - sunrise_with_prediction_df.predicted_mean
    )

    sunrise_with_prediction_df["Brand_Name"] = sunrise_with_prediction_df.Brand_Name[1]

    print(stlf_res.summary())

    mean_error_percentage = sum(
        abs(
            sunrise_with_prediction_df.residuals.dropna()
            / sunrise_with_prediction_df.rev_net_total.dropna()
        )
    ) / len(sunrise_with_prediction_df.residuals.dropna())

    print(f"Mean Absolute Error: {round(mean_error_percentage*100,3)}%")

    return sunrise_with_prediction_df

In [61]:
def plot_presentation(
    sunrise_with_prediction_df: pd.DataFrame, brand_name: str
) -> None:
    _date_filter_ = pd.to_datetime("2012-01-01", format="%Y-%m-%d")
    _filter_ = sunrise_with_prediction_df.index >= _date_filter_
    plot_df = sunrise_with_prediction_df[_filter_]
    # brand_name

    fig = make_subplots(
        rows=2,
        cols=1,
        subplot_titles=("Observations & Training & Forecast (CHF)", "Residuals (%)"),
        vertical_spacing=0.05,
        horizontal_spacing=0.02,
        shared_xaxes=True,
        row_heights=[0.7, 0.3],
    )

    ## Confidence Intervals: lower band
    fig.add_trace(
        go.Scatter(
            x=plot_df.index,
            y=plot_df.lower,  # * plot_df.days_in_month,
            name="lower",
            mode="lines",
            line_shape="spline",
            fill=None,
            showlegend=False,
            line=dict(color="white"),
            opacity=0.25,
        ),
        row=1,
        col=1,
    )

    ## Confidence Intervals: Upper band
    fig.add_trace(
        go.Scatter(
            x=plot_df.index,
            y=plot_df.upper,  # * plot_df.days_in_month,
            name="upper",
            mode="lines",
            line_shape="spline",
            fill="tonexty",  # fill area between trace0 and trace1
            showlegend=False,
            line=dict(color="white"),
            opacity=0.25,
        ),
        row=1,
        col=1,
    )

    ## Observations:
    fig.add_trace(
        go.Bar(
            x=plot_df.index,
            y=plot_df.rev_net_total,
            name="Net Revenue (monthly)",
            opacity=0.5,
            # mode="markers",
            # line=dict(color="black"),
        ),
        row=1,
        col=1,
    )

    ## Model IN-sample predictions:
    subplot_df = plot_df[plot_df.Calendar_Year_Month_Id <= upper_training_limit]
    fig.add_trace(
        go.Scatter(
            x=subplot_df.index,
            y=subplot_df.predicted_mean,  # * subplot_df.days_in_month,
            name="Estimation",
            mode="lines+markers",
            line_shape="spline",
        ),
        row=1,
        col=1,
    )

    ## Model OUT-sample Predictions:
    subplot_df = plot_df[plot_df.Calendar_Year_Month_Id > upper_training_limit]
    fig.add_trace(
        go.Scatter(
            x=subplot_df.index,
            y=subplot_df.predicted_mean,  # * subplot_df.days_in_month,
            name="Forecast",
            mode="lines+markers",
        ),
        row=1,
        col=1,
    )

    ## Residuals: Positive, over estimated
    subplot_df = plot_df[plot_df.residuals > 0]
    fig.add_trace(
        go.Bar(
            x=subplot_df.index,
            y=subplot_df.residuals / subplot_df.rev_net_total * 100,
            name="Residuals (+)",
            legendgroup="resis",
            showlegend=False,
        ),
        row=2,
        col=1,
    )

    ## Residuals: Negative, under-estimated
    subplot_df = plot_df[plot_df.residuals <= 0]
    fig.add_trace(
        go.Bar(
            x=subplot_df.index,
            y=subplot_df.residuals / subplot_df.rev_net_total * 100,
            name="Residuals (-)",
            legendgroup="resis",
            showlegend=False,
        ),
        row=2,
        col=1,
    )

    _date_vlines_ = pd.Series(
        pd.date_range(
            start=plot_df.index.min() - pd.DateOffset(years=1),
            end=plot_df.index.max() + pd.DateOffset(years=1),
            freq="Y",
        )
    ) - pd.DateOffset(days=15)

    for _i_ in [1, 2]:
        for _x_ in _date_vlines_:
            fig.add_vline(
                x=_x_,
                line_dash="dot",
                row=_i_,
                col=1,
                line=dict(color="grey"),
                # annotation_text="Jan 1, 2018 baseline",
                # annotation_position="bottom right",
            )

    ## Configs
    fig.update_layout(
        title_text=f"{brand_name} Prepaid Monthly Net-Revenues: STD based on SARIMA & LOESS",
        margin=dict(l=20, r=20, t=50, b=20),
        hovermode="x unified",
    )
    _link_to_file_ = ANALYSIS_DIR / f"FIA0590_{brand_name}_with_forecast.html"
    fig.write_html(_link_to_file_)

    print(_link_to_file_)

In [62]:
sunrise_with_prediction_df = create_forecasting_df(
    sunrise_df=sunrise_df, upper_training_limit=202109
)

print("\b")
plot_presentation(
    sunrise_with_prediction_df=sunrise_with_prediction_df, brand_name="Sunrise"
)

                    STL Decomposition and SARIMAX Results                     
Dep. Variable:                      y   No. Observations:                  129
Model:                 ARIMA(1, 2, 1)   Log Likelihood               -1686.261
Date:                Tue, 15 Feb 2022   AIC                           3378.521
Time:                        14:22:43   BIC                           3387.054
Sample:                    01-01-2011   HQIC                          3381.988
                         - 09-01-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1926      0.159      1.209      0.227      -0.120       0.505
ma.L1         -0.2359      0.167     -1.413      0.158      -0.563       0.091
sigma2      5.418e+09   2.53e-11   2.15e+20      0.0

In [63]:
yallo_with_prediction_df = create_forecasting_df(
    sunrise_df=yallo_df, upper_training_limit=202109
)

print("\b")
plot_presentation(
    sunrise_with_prediction_df=yallo_with_prediction_df, brand_name="Yallo"
)

                    STL Decomposition and SARIMAX Results                     
Dep. Variable:                      y   No. Observations:                  129
Model:                 ARIMA(1, 2, 1)   Log Likelihood               -1687.725
Date:                Tue, 15 Feb 2022   AIC                           3381.451
Time:                        14:22:44   BIC                           3389.983
Sample:                    01-01-2011   HQIC                          3384.917
                         - 09-01-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.8832      0.017     51.407      0.000       0.850       0.917
ma.L1         -0.9999      0.051    -19.613      0.000      -1.100      -0.900
sigma2      1.088e+10   4.71e-12   2.31e+21      0.0

In [64]:
lebara_with_prediction_df = create_forecasting_df(
    sunrise_df=lebara_df, upper_training_limit=202109
)

print("\b")
plot_presentation(
    sunrise_with_prediction_df=lebara_with_prediction_df, brand_name="Lebara"
)

                    STL Decomposition and SARIMAX Results                     
Dep. Variable:                      y   No. Observations:                  129
Model:                 ARIMA(1, 2, 1)   Log Likelihood               -1725.555
Date:                Tue, 15 Feb 2022   AIC                           3457.109
Time:                        14:22:45   BIC                           3465.642
Sample:                    01-01-2011   HQIC                          3460.576
                         - 09-01-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.8402      0.025     33.291      0.000       0.791       0.890
ma.L1         -0.9998      0.049    -20.560      0.000      -1.095      -0.905
sigma2      2.241e+10   2.21e-12   1.01e+22      0.0

# Deployment

In [65]:
output_df = pd.concat(
    [sunrise_with_prediction_df, yallo_with_prediction_df, lebara_with_prediction_df]
)

output_df["Calendar_Year_Month_Date"] = output_df.index

_cols_ = [
    "Calendar_Year_Month_Date",
    "Brand_Name",
    "rev_net_total",
    "predicted_mean",
    "residuals",
    "lower",
    "upper",
]
output_df = (
    output_df[_cols_]
    .reset_index()
    .drop(columns="index")
    .rename(columns={"predicted_mean": "stl_estimatation"})
)

output_df = output_df[
    output_df.Calendar_Year_Month_Date
    >= pd.to_datetime("2012-01-01", format="%Y-%m-%d")
]

prediction_flag = output_df.Calendar_Year_Month_Date > pd.to_datetime(
    "2021-09-01", format="%Y-%m-%d"
)

output_df["residuals_percentage"] = output_df.residuals / output_df.rev_net_total * 100
output_df["prediction_flag"] = prediction_flag

In [35]:
output_df.to_excel(
    ANALYSIS_DIR / "FIA0590_prepaid_forecasts_20220208.xlsx",
    index=False,
)